# Contraceptive Method Choice

In [1]:
import os
import pandas as pd


### Import dataset

**original dataset info**

   1. Wife's age                     (numerical)
   2. Wife's education               (categorical)      1=low, 2, 3, 4=high
   3. Husband's education            (categorical)      1=low, 2, 3, 4=high
   4. Number of children ever born   (numerical)
   5. Wife's religion                (binary)           0=Non-Islam, 1=Islam
   6. Wife's now working?            (binary)           0=Yes, 1=No
   7. Husband's occupation           (categorical)      1, 2, 3, 4
   8. Standard-of-living index       (categorical)      1=low, 2, 3, 4=high
   9. Media exposure                 (binary)           0=Good, 1=Not good
   10. Contraceptive method used     (class attribute)  1=No-use, 2=Long-term, 3=Short-term **<=**
   
**_Notes_**
- Column #10 is to be predicted.
- I'm going to change columns names and invert some of the values, as they don't correspond to common notation (e.g. #5,6,9)

In [2]:
path_to_data_set = os.path.join("UCI_datasets", "cmc", "cmc.data")

names = {
    "W_AGE"                  : 'int8',
    "W_EDU"                  : 'category',
    "H_EDU"                  : 'category',
    "NUM_CHILDREN"           : 'int8',
    "W_ISLAM"                : 'bool',
    "W_WORKING"              : 'bool',
    "H_JOB"                  : 'category',
    "SOL"                    : 'category',
    "BAD_MEDIA"              : 'bool',
    "CONT_USAGE"             : 'category'
}

table = pd.read_table(path_to_data_set, delimiter=",",  
                      header=None, index_col=False, 
                      names=names.keys(),
                      dtype=names,
                      engine='python'
                     )

In [3]:
table.head()

,W_AGE,W_EDU,H_EDU,NUM_CHILDREN,W_ISLAM,W_WORKING,H_JOB,SOL,BAD_MEDIA,CONT_USAGE
0,24,2,3,3,True,True,2,3,False,1
1,45,1,3,10,True,True,3,4,False,1
2,43,2,3,7,True,True,3,4,False,1
3,42,3,2,9,True,True,3,3,False,1
4,36,3,3,8,True,True,3,2,False,1


inverting values of two columns

In [4]:
table["W_WORKING"] = table["W_WORKING"].apply(lambda x: not x)
# table["W_ISLAM"] = table["W_ISLAM"].apply(lambda x: not x)

In [5]:
#### Data visualization

In [6]:
for x in table.columns:
    print(x)
    print(table[x].unique())

W_AGE
[24 45 43 42 36 19 38 21 27 44 26 48 39 37 46 40 29 31 33 25 28 47 32 49
 34 20 22 30 23 35 41 17 18 16]
W_EDU
[2, 1, 3, 4]
Categories (4, object): [2, 1, 3, 4]
H_EDU
[3, 2, 4, 1]
Categories (4, object): [3, 2, 4, 1]
NUM_CHILDREN
[ 3 10  7  9  8  0  6  1  2  4  5 12 11 13 16]
W_ISLAM
[ True False]
W_WORKING
[False  True]
H_JOB
[2, 3, 1, 4]
Categories (4, object): [2, 3, 1, 4]
SOL
[3, 4, 2, 1]
Categories (4, object): [3, 4, 2, 1]
BAD_MEDIA
[False  True]
CONT_USAGE
[1, 2, 3]
Categories (3, object): [1, 2, 3]


#### Data visualization

### Train model

### Model evaluation 